In [3]:
# Dependencies and Setup
import pandas as pd

# Loading Files
school_csv = "Resources/schools_complete.csv"
student_csv = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_csv)
student_data = pd.read_csv(student_csv)

# Combine the data into a single dataset.  
school_full = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [4]:
school_full.head(1)

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635


In [151]:
### DISTRICT SUMMARY
schools_count = len(school_full["school_name"].value_counts())
school_count = len(school_data)
print(school_count)

stu_count = school_full["Student ID"].count()
student_count = "{:,}".format(len(student_data))
print(student_count)

total_budget = "${:,.2f}".format(school_data["budget"].sum())
print(total_budget)

avg_math = school_full["math_score"].mean()
print(avg_math)

avg_read = school_full["reading_score"].mean()
print(avg_read)

count_pass_math = sum(x >=70 for x in school_full["math_score"])
perc_pass_math = "{:.2%}".format(count_pass_math/stu_count)
print(perc_pass_math)

count_pass_read = sum(x >=70 for x in school_full["reading_score"])
perc_pass_read = "{:.2%}".format(count_pass_read/stu_count)
print(perc_pass_read)

overall_pass = school_full.loc[(school_full["math_score"]>=70)&(school_full["reading_score"]>=70)]
perc_overall_pass = "{:.2%}".format((len(overall_pass)/stu_count))
print(perc_overall_pass)

dist_data = [{"Total Schools": school_count,"Total Students": stu_count,"Total Budget": total_budget,
              "Average Math Score": avg_math,"Average Reading Score": avg_read,
             "% Passing Math": perc_pass_math,"% Passing Reading": perc_pass_read,
             "% Overall Passing": perc_overall_pass}]

15
39,170
$24,649,428.00
78.98537145774827
81.87784018381414
74.98%
85.81%
65.17%


In [152]:
### DISTRICT SUMMARY DATAFRAME
dist_data
district_df = pd.DataFrame(dist_data)
display(district_df)

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,"$24,649,428.00",78.985371,81.87784,74.98%,85.81%,65.17%


In [282]:
### SCHOOL SUMMARY
school = school_full.groupby(['school_name'])


#names = list(school["school_name"].unique())
#print(names)

school_data.sort_values("school_name",axis=0,ascending=True,inplace=True)
names = list(school_data["school_name"])

types = list(school_data["type"])
#print(types)

#budg = school["budget"].apply(lambda x: "${:,.2f}".format(x))
school_budget = list(school_data["budget"])
budget = []
for i in range(len(names)):
    ttl_bdg = "${:,.2f}".format(school_budget[i])
    budget.append(ttl_bdg)

#budg = "${:,.2f}".format(school_budget)
print(budget)
print(school_budget)

budget_per_stud = []
for i in range(len(names)):
    budg = "${:,.2f}".format(school_budget[i]/count[i])
    budget_per_stud.append(budg)
#print(budget_per_stud)

school_stu_count = list(school["school_name"].size())
#print(school_stu_count)
count = list(school_stu_count)

avg_school_math = list(school["math_score"].mean())
#print(avg_school_math)

avg_school_read = list(school["reading_score"].mean())
#print(avg_school_read)

school_count_pass_math = school.apply(lambda x: x[x['math_score'] >= 70 ]['Student ID'].count())
math = list(school_count_pass_math)
#print(math)

school_perc_math = []
for i in range(len(count)):
    perc = "{:.2%}".format(math[i]/count[i])
    school_perc_math.append(perc)
#print(school_perc_math)

school_count_pass_read = school.apply(lambda x: x[x['reading_score'] >= 70]['Student ID'].count())
read = list(school_count_pass_read)
#print(read)

school_perc_read = []
for i in range(len(count)):
    perc = "{:.2%}".format(read[i]/count[i])
    school_perc_read.append(perc)
#print(school_perc_read)

school_pass = []
school_pass_overall = school.apply(lambda x: x[(x['math_score'] >= 70)&(x['reading_score'] >= 70)]['Student ID'].count())
overall = list(school_pass_overall)
for i in range(len(count)):
    perc = "{:.2%}".format(overall[i]/count[i])
    school_pass.append(perc)
#print(school_pass)


school_tbl = {"School Type":types,"Total Students": count,"Total School Budget": budget,
              "Per Student Budget": budget_per_stud,"Average Math Score": avg_school_math,"Average Reading Score": avg_school_read,
             "% Passing Math": school_perc_math,"% Passing Reading": school_perc_read,
             "% Overall Passing": school_pass}
#school_tbl

['$3,124,928.00', '$1,081,356.00', '$1,884,411.00', '$1,763,916.00', '$917,500.00', '$3,022,020.00', '$248,087.00', '$1,910,635.00', '$3,094,650.00', '$585,858.00', '$2,547,363.00', '$1,056,600.00', '$1,043,130.00', '$1,319,574.00', '$1,049,400.00']
[3124928, 1081356, 1884411, 1763916, 917500, 3022020, 248087, 1910635, 3094650, 585858, 2547363, 1056600, 1043130, 1319574, 1049400]


In [283]:
### SCHOOLS SUMMARY DATAFRAME

school_df = pd.DataFrame(school_tbl)
school_df = school_df.set_index(pd.Index(names))
display(school_df)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.68%,81.93%,54.64%
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.13%,97.04%,91.33%
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.99%,80.74%,53.20%
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.31%,79.30%,54.29%
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.39%,97.14%,90.60%
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.75%,80.86%,53.53%
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.51%,96.25%,89.23%
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.68%,81.32%,53.51%
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.06%,81.22%,53.54%
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.59%,95.95%,90.54%
